In [1]:
import os
import sys
import glob
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt

from scipy.stats import rayleigh
from scipy.stats import ks_2samp
from numpy import linspace
from numpy.random import choice
from networkx import Graph

from learning_dist_metrics.ldm import LDM
from learning_dist_metrics.dist_metrics import weighted_euclidean

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
user_df = pd.read_csv("data/drhuang_person.csv", header=0)
friend_df = pd.read_csv("data/drhuang_friend.csv", header=0,)

## split data by column
uid_df = user_df["ID"]
profile_df = user_df[[c for c in user_df.columns if c != "ID"]]

friend_df.head(3)
#friend_df.columns = [""range(0, friend_df.shape[1]-1)
relation_mtx = friend_df.ix[:, 1:]
relation_mtx.head(4)

friend_pairs = []
uid_row = friend_df.ix[:, 0]
uid_col = uid_row

# convert relationship matrix into a list of user pairs
# to represent friendships, iterating through items
# on the lower triangle of the matrix
for i, uid_a in enumerate(uid_row):
    for j, uid_b in enumerate(uid_col):

        if uid_a < uid_b:
            ind = relation_mtx.ix[i, j]
            if ind == 1:
                friend_pairs.append((uid_a, uid_b))

# load relationship into networkx.Graph()
from networkx import Graph

fnx = Graph()
fnx.add_edges_from(friend_pairs)

def scalar(x):
    _min, _max = min(x), max(x)
    _range = (_max - _min) * 1.0
    res = [(i - _min) / _range for i in x]
    return res

## transform data
new_profile_df = pd.DataFrame()

new_profile_df["ID"]        = uid_df #uid_df["ID"]
new_profile_df["Gender"]    = profile_df.ix[:, 0]
new_profile_df["Age"]       = scalar(profile_df["Age"])
new_profile_df["Region"]    = scalar(profile_df["Region"])
new_profile_df["Education"] = scalar(profile_df["Education"])
new_profile_df["Income"]    = scalar(profile_df["Income"])
new_profile_df["Hobby"]     = scalar(profile_df["Hobby"])
new_profile_df["Duration"]  = scalar(profile_df["Duration"])

In [3]:
from GWDLearner import *

# w = ldm_train_with_list(all_uids, new_profile_df, friend_pairs, retain_type=1)
res = learning_wrapper(profile_df=new_profile_df, friends_pair=friend_pairs, max_iter=50,\
                       k=1, c=0.1, dropout_rate=0.2, fit_rayleigh=True, verbose=True, \
                       buffer_group_enabled=True)

Initiating ...
1 iteration is in processing ...
--- 52.17 seconds ---
1) #fit: 88, #unfit: 12, #buffer: 0
1) #fit: 88, #unfit: 12, #buffer: 0
1) #fit: 88, #unfit: 0, #buffer: 12
2 iteration is in processing ...
--- 41.00 seconds ---
1) #fit: 79, #unfit: 9, #buffer: 12
1) #fit: 91, #unfit: 9, #buffer: 0
1) #fit: 91, #unfit: 0, #buffer: 9
3 iteration is in processing ...
--- 43.56 seconds ---
1) #fit: 85, #unfit: 6, #buffer: 9
1) #fit: 94, #unfit: 6, #buffer: 0
1) #fit: 94, #unfit: 0, #buffer: 6


UnboundLocalError: local variable 'user_dropout_dict' referenced before assignment